In [1]:


import keras
from keras import Sequential, regularizers
from keras.layers import Dense, BatchNormalization, Dropout, TimeDistributed, LSTM, LayerNormalization
from scipy.stats import mode
import pandas
import numpy as np

# Training parameters.
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from keras.utils.np_utils import to_categorical

# from layer_norm import LayerNorm1D
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(
    physical_devices[0],True)
# run_opts = tf.RunOptions(report_tensor_allocations_upon_oom = True)


batch_size = 1 # 16 no good
num_classes = 4
epochs = 20
first_dim = 40
# Number of features
feature_idx = 11
print('loading data')
dataframe = pandas.read_csv("dataset/combined_data/2nd_labeled.csv", header=None)
# dataframe = pandas.read_csv("dataset/combined_data/test.csv", header=None)

loading data


In [2]:
dataset = dataframe.values
print('data loaded')
print('Making Train Validation Test Set')
datasetlen=len(dataset)
# print(datasetlen % 10)
if datasetlen % first_dim is not 0:
    datasetlen = datasetlen-datasetlen % first_dim
X = np.array([dataset[i:i+first_dim, 0:feature_idx].astype('float32') for i in range(0, datasetlen, first_dim)])
Y = np.array([mode(dataset[i:i+first_dim,feature_idx:].astype(int))[0][0][0] for i in range(0, datasetlen, first_dim)])

#cann't do this because it will split all the patterns in car
#print(X[0])
print(set(Y))
X=X.reshape(X.shape[0], first_dim, feature_idx, 1)
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.20, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.05, random_state=42)
X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
X_test = X_test.astype('float32')

print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)
print(y_test.shape)

y_train = to_categorical(y_train, num_classes)
y_val = to_categorical(y_val, num_classes)
y_test = to_categorical(y_test, num_classes)

<>:6: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:6: SyntaxWarning: "is not" with a literal. Did you mean "!="?
C:\Users\shahr\AppData\Local\Temp/ipykernel_91216/3646515424.py:6: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if datasetlen % first_dim is not 0:


data loaded
Making Train Validation Test Set
{0, 1, 2, 3}
(78375, 40, 11, 1)
(78375,)
(20625, 40, 11, 1)
(20625,)
(4125, 40, 11, 1)
(4125,)


In [3]:
print('Making Model')
model=Sequential()
model.add(Dense(256, activation='relu',kernel_regularizer=regularizers.l2(0.0001), input_shape=(first_dim, feature_idx,1)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu',kernel_regularizer=regularizers.l2(0.0001)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu',kernel_regularizer=regularizers.l2(0.0001)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(TimeDistributed(LSTM(512,kernel_regularizer=regularizers.l2(0.0001))))
model.add(LayerNormalization())
model.add(LSTM(512,kernel_regularizer=regularizers.l2(0.0001)))
model.add(Dense(num_classes, activation='softmax',kernel_regularizer=regularizers.l2(0.0001)))
model.summary()
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

Making Model
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 40, 11, 256)       512       
_________________________________________________________________
batch_normalization (BatchNo (None, 40, 11, 256)       1024      
_________________________________________________________________
dropout (Dropout)            (None, 40, 11, 256)       0         
_________________________________________________________________
dense_1 (Dense)              (None, 40, 11, 512)       131584    
_________________________________________________________________
batch_normalization_1 (Batch (None, 40, 11, 512)       2048      
_________________________________________________________________
dropout_1 (Dropout)          (None, 40, 11, 512)       0         
_________________________________________________________________
dense_2 (Dense)              (None, 40, 11,

In [4]:
# # Training.
# history_callback = model.fit(X_train, y_train,
#           batch_size=batch_size,
#           epochs=epochs,
#           verbose=1,
#           validation_data=(X_val, y_val))

# model.save('CAN_RNN_model_1line_2021.h5')

In [5]:
model.load_weights('CAN_RNN_model_1line_2021.h5')


Y_test = np.argmax(y_test, axis=1)
y_pred = model.predict_classes(X_test)

print("\n", classification_report(Y_test, y_pred,digits=4))

C:\ProgramData\Miniconda3\envs\TF-2.5\lib\site-packages\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '



               precision    recall  f1-score   support

           0     0.9864    0.9997    0.9930      3766
           1     1.0000    1.0000    1.0000       129
           2     1.0000    0.9722    0.9859        72
           3     0.9908    0.6835    0.8090       158

    accuracy                         0.9872      4125
   macro avg     0.9943    0.9139    0.9470      4125
weighted avg     0.9872    0.9872    0.9861      4125



In [6]:
# scores = model.evaluate(X_test, y_test, verbose=0)
# print('Test loss:', scores[0])
# print('Test accuracy:', scores[1])


import sys
sys.path.append("E:\\Project\\Kari_anomaly_2\\Adversarial_attacks\\cleverhans")
from cleverhans.tf2.attacks.fast_gradient_method import fast_gradient_method
from cleverhans.tf2.attacks.projected_gradient_descent import projected_gradient_descent


class flags_struct:
    def __init__(self,
                 nb_epochs: int,
                 runs: int,
                 eps: float,
                 adv_train: bool,
                 is_categorical: bool,
                 batch_size: int,
                 UCR_datasets: list):
        self.nb_epochs = nb_epochs
        self.runs = runs
        self.eps = eps
        self.adv_train = adv_train
        self.is_categorical = is_categorical
        self.batch_size = batch_size
        self.UCR_datasets = UCR_datasets


FLAGS = flags_struct(nb_epochs=2000,
                     runs=3,
                     eps=0.1,
                     adv_train=False,
                     is_categorical=False,
                     batch_size=1,
                     UCR_datasets=['50words'])

In [14]:
X_FSGM_list=[]
# for 
X_FSGM_list.append(fast_gradient_method(model, X_test[0:400,:,:,:], FLAGS.eps, np.inf,y=tf.argmax(tf.cast(y_test[0:400], dtype=tf.int32),1)))
X_FSGM_list.append(fast_gradient_method(model, X_test[400:800,:,:,:], FLAGS.eps, np.inf,y=tf.argmax(tf.cast(y_test[400:800], dtype=tf.int32),1)))
X_FSGM_list.append(fast_gradient_method(model, X_test[800:1200,:,:,:], FLAGS.eps, np.inf,y=tf.argmax(tf.cast(y_test[800:1200], dtype=tf.int32),1)))
X_FSGM_list.append(fast_gradient_method(model, X_test[1200:1600,:,:,:], FLAGS.eps, np.inf,y=tf.argmax(tf.cast(y_test[1200:1600], dtype=tf.int32),1)))
X_FSGM_list.append(fast_gradient_method(model, X_test[1600:2000,:,:,:], FLAGS.eps, np.inf,y=tf.argmax(tf.cast(y_test[1600:2000], dtype=tf.int32),1)))
X_FSGM_list.append(fast_gradient_method(model, X_test[2000:2400,:,:,:], FLAGS.eps, np.inf,y=tf.argmax(tf.cast(y_test[2000:2400], dtype=tf.int32),1)))
X_FSGM_list.append(fast_gradient_method(model, X_test[2400:2800,:,:,:], FLAGS.eps, np.inf,y=tf.argmax(tf.cast(y_test[2400:2800], dtype=tf.int32),1)))
X_FSGM_list.append(fast_gradient_method(model, X_test[2800:3200,:,:,:], FLAGS.eps, np.inf,y=tf.argmax(tf.cast(y_test[2800:3200], dtype=tf.int32),1)))
X_FSGM_list.append(fast_gradient_method(model, X_test[3200:3600,:,:,:], FLAGS.eps, np.inf,y=tf.argmax(tf.cast(y_test[3200:3600], dtype=tf.int32),1)))
X_FSGM_list.append(fast_gradient_method(model, X_test[3600:4000,:,:,:], FLAGS.eps, np.inf,y=tf.argmax(tf.cast(y_test[3600:4000], dtype=tf.int32),1)))
X_FSGM_list.append(fast_gradient_method(model, X_test[4000:4125,:,:,:], FLAGS.eps, np.inf,y=tf.argmax(tf.cast(y_test[4000:4125], dtype=tf.int32),1)))


X_FSGM=np.concatenate(X_FSGM_list,axis=0)
print(X_FSGM.shape)
y_FSGM = model.predict_classes(X_FSGM)

print("\n", classification_report(Y_test, y_FSGM,digits=4))

# scores = model.evaluate(X_test, y_test, verbose=0)
# print('Test loss:', scores[0])
# print('Test accuracy:', scores[1])

(4125, 40, 11, 1)

               precision    recall  f1-score   support

           0     0.9191    0.1901    0.3151      3766
           1     0.0000    0.0000    0.0000       129
           2     0.0183    0.8472    0.0357        72
           3     0.0000    0.0000    0.0000       158

    accuracy                         0.1884      4125
   macro avg     0.2343    0.2593    0.0877      4125
weighted avg     0.8395    0.1884    0.2883      4125



C:\ProgramData\Miniconda3\envs\TF-2.5\lib\site-packages\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
C:\ProgramData\Miniconda3\envs\TF-2.5\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Miniconda3\envs\TF-2.5\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Pr

In [15]:
# X_PGD = projected_gradient_descent(model, X_test, FLAGS.eps, 0.01, 40, np.inf, y=tf.argmax(tf.cast(y_test, dtype=tf.int32),1))
# 
# y_PGD = model.predict_classes(X_PGD)
# 
# print("\n", classification_report(Y_test, y_PGD,digits=4))

X_PGD_list=[]
# for 
X_PGD_list.append(projected_gradient_descent(model, X_test[0:400,:,:,:], FLAGS.eps, 0.01, 40, np.inf,y=tf.argmax(tf.cast(y_test[0:400], dtype=tf.int32),1)))
X_PGD_list.append(projected_gradient_descent(model, X_test[400:800,:,:,:], FLAGS.eps, 0.01, 40,  np.inf,y=tf.argmax(tf.cast(y_test[400:800], dtype=tf.int32),1)))
X_PGD_list.append(projected_gradient_descent(model, X_test[800:1200,:,:,:], FLAGS.eps, 0.01, 40, np.inf,y=tf.argmax(tf.cast(y_test[800:1200], dtype=tf.int32),1)))
X_PGD_list.append(projected_gradient_descent(model, X_test[1200:1600,:,:,:], FLAGS.eps, 0.01, 40, np.inf,y=tf.argmax(tf.cast(y_test[1200:1600], dtype=tf.int32),1)))
X_PGD_list.append(projected_gradient_descent(model, X_test[1600:2000,:,:,:], FLAGS.eps, 0.01, 40, np.inf,y=tf.argmax(tf.cast(y_test[1600:2000], dtype=tf.int32),1)))
X_PGD_list.append(projected_gradient_descent(model, X_test[2000:2400,:,:,:], FLAGS.eps, 0.01, 40, np.inf,y=tf.argmax(tf.cast(y_test[2000:2400], dtype=tf.int32),1)))
X_PGD_list.append(projected_gradient_descent(model, X_test[2400:2800,:,:,:], FLAGS.eps, 0.01, 40, np.inf,y=tf.argmax(tf.cast(y_test[2400:2800], dtype=tf.int32),1)))
X_PGD_list.append(projected_gradient_descent(model, X_test[2800:3200,:,:,:], FLAGS.eps, 0.01, 40, np.inf,y=tf.argmax(tf.cast(y_test[2800:3200], dtype=tf.int32),1)))
X_PGD_list.append(projected_gradient_descent(model, X_test[3200:3600,:,:,:], FLAGS.eps, 0.01, 40, np.inf,y=tf.argmax(tf.cast(y_test[3200:3600], dtype=tf.int32),1)))
X_PGD_list.append(projected_gradient_descent(model, X_test[3600:4000,:,:,:], FLAGS.eps, 0.01, 40, np.inf,y=tf.argmax(tf.cast(y_test[3600:4000], dtype=tf.int32),1)))
X_PGD_list.append(projected_gradient_descent(model, X_test[4000:4125,:,:,:], FLAGS.eps, 0.01, 40, np.inf,y=tf.argmax(tf.cast(y_test[4000:4125], dtype=tf.int32),1)))


X_PGD=np.concatenate(X_PGD_list,axis=0)
print(X_PGD.shape)
y_PGD = model.predict_classes(X_PGD)

print("\n", classification_report(Y_test, y_PGD,digits=4))

(4125, 40, 11, 1)

               precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000    3766.0
           1     0.0000    0.0000    0.0000     129.0
           2     0.0000    0.0000    0.0000      72.0
           3     0.0000    0.0000    0.0000     158.0

    accuracy                         0.0000    4125.0
   macro avg     0.0000    0.0000    0.0000    4125.0
weighted avg     0.0000    0.0000    0.0000    4125.0



C:\ProgramData\Miniconda3\envs\TF-2.5\lib\site-packages\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
